In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import tensorflow

In [17]:
#----------------------------------------label encoding------------------------------------------------------------------------------
data = pd.read_csv("/Users/derri/Onedrive/ml/train.csv")
sf = pd.Categorical(data.SexuponOutcome)
sf #we can see that there are five categories 
sex_mapping={"Intact Male":"0","Neutered Male":"0","Intact Female":"1","Spayed Female":"1","Unknown":"2"} 
data["sex"]="1" #create a new feature, sex 
data.sex=data.SexuponOutcome.map(sex_mapping)
fertility_mapping={"Intact Male":"0","Intact Female":"0","Neutered Male":"1","Spayed Female":"1","Unknown":"2"}
data["fertility"]="1" #create a new feature, fertility
data.fertility=data.SexuponOutcome.map(fertility_mapping)

color=pd.Categorical(data.Color) 
color #there are 366 colors
data["MixColor"]=pd.Series(data.Color.str.contains('/')).astype('int') #create a new feature, MixColor: if mix, then 1, otherwise, 0
data["color"]=data.Color.str.split("/")
data.color=[data.color[i][0] for i in range(len(data.color))]

ColorCounts=data.color.value_counts()
color_mapping={"Black": "0" , "White":"1", "Brown Tabby": "2" , "Brown": "3" , "Tan": "4" , "Orange Tabby": "5" ,
               "Blue": "6" ,"Tricolor": "7" ,"Red": "8" , "Brown Brindle": "9" , "Blue Tabby": "10" ,
               "Tortie":"11", "Calico": "12","Chocolate": "13" , "Torbie":"14", "Sable": "15" ,
               "Cream Tabby": "16", "Buff":"17", "Yellow": "18" , "Gray": "19" ,"Cream":"20",
               "Fawn": "21", "Lynx Point": "22" , "Blue Merle": "23" , "Seal Point": "24" ,"Black Brindle": "25" ,
               "Flame Point":"26" , "Gold": "27", "Brown Merle": "28" , "Black Smoke": "29" ,"Black Tabby":"30" ,
               "Silver": "31" , "Red Merle": "32" , "Gray Tabby": "33", "Blue Tick": "34" ,"Orange": "35",
               "Silver Tabby": "36" , "Red Tick": "37", "Lilac Point": "38" , "Tortie Point": "39","Yellow Brindle": "40" ,
               "Blue Point": "41" , "Calico Point": "42" , "Apricot": "43","Chocolate Point": "44" , "Blue Cream": "45" ,
               "Liver": "46" , "Blue Tiger": "47" , "Blue Smoke": "48","Liver Tick": "49", "Brown Tiger": "50" ,
               "Black Tiger": "51" , "Agouti": "52" , "Silver Lynx Point": "53", "Orange Tiger": "54", "Ruddy": "55" , "Pink": "56"}
#create a new feature: colorC, what is the encoding color
data["colorC"]="1"
data.colorC=data.color.map(color_mapping)

In [18]:
data.AgeuponOutcome.value_counts()
data["agenumber"]=data.AgeuponOutcome.str.split(expand=True)[0].astype("float")
data["ageperiod"]=data.AgeuponOutcome.str.split(expand=True)[1]

#create a new feature, age, age by day
ageD=[]
for i in range(len(data.AnimalID)):
    if data.ageperiod[i]=="day" or data.ageperiod[i]=="days":
        ageD.append(1*data.agenumber[i])
    elif data.ageperiod[i]=="week" or data.ageperiod[i]=="weeks":
        ageD.append(7*data.agenumber[i])
    elif data.ageperiod[i]=="month" or data.ageperiod[i]=="months":
        ageD.append(30*data.agenumber[i])
    elif data.ageperiod[i]=="year" or data.ageperiod[i]=="years":
        ageD.append(365*data.agenumber[i])
    else: ageD.append("NaN")
data["ageD"]=ageD

ageY=[]#create a new feature, ageY, that is the age classifier
for i in range(len(data.AnimalID)):
    if data.ageperiod[i]=="year" or data.ageperiod[i]=="years":
        ageY.append(data.agenumber[i].astype("int"))
    elif data.ageD[i]=="NaN":
        ageY.append("NaN")
    else: ageY.append(0)
data["ageY"]=ageY

In [19]:
data["MainBreed"]=data.Breed.str.split("/") #create a new variable, MainBreed
data.MainBreed=[data.MainBreed[i][0] for i in range(len(data.MainBreed))]

#create a new variable, MixBreed 
data["MixBreed"]=pd.Series(data.Breed.str.contains('Mix')|data.Breed.str.contains('/')).astype("int") 
class_le = LabelEncoder() 
data.MainBreed = class_le.fit_transform(data.MainBreed)#fit and transform the class  
# dog:1 cat:2 
animal_mapping={"Dog":"1","Cat":"2"} #create a new feature, animal 
data["animal"]="1" 
data.animal=data.AnimalType.map(animal_mapping)  #create a new feature, HaveName 
data["NameC"]=abs(pd.Series(data.Name.isnull()).astype("int")-1)

data.OutcomeType.value_counts()
outcometype_mapping={"Adoption":"0","Transfer":"1","Return_to_owner":"2","Euthanasia":"3","Died":"4"}
#create a new target: outcome
data["outcome"]="1"
data.outcome=data.OutcomeType.map(outcometype_mapping)

In [10]:
list = data[(data.ageD=="NaN")].index.tolist()
data = data.drop(list) #remove 18 rows

data.sex = data.sex.fillna('nan')
list = data[(data.sex=='nan')].index.tolist()
data = data.drop(list) #remove 1 row
data.to_csv("c")

In [22]:
data1 = pd.read_csv("/Users/derri/Onedrive/ml/train1.csv")
data1.columns
data1_copy = data1.copy(deep=True)
data1_copy = data1_copy.drop(columns=[ 'sex', 'fertility',
       'MixColor', 'colorC',  'ageY',
       'MixBreed', 'animal', 'NameC','MainBreedMix','Main Breed'])

data1 = data1.drop(columns=['Aggressive', 'At Vet', 'Barn', 'Behavior',
       'Court/Investigation', 'Enroute', 'Foster', 'In Foster', 'In Kennel',
       'In Surgery', 'Medical', 'Offsite', 'Partner', 'Rabies Risk', 'SCRP',
       'Suffering','BreedName','color','agenumber', 'ageperiod', 'ageD','outcome','outcomeG','MainBreedMix'])

data1.iloc[:,1:10] = data1.iloc[:,1:10].astype("int")
data1.iloc[:,1:10] = data1.iloc[:,1:10].astype("object")
data1_onehot = pd.get_dummies(data1.iloc[:,1:10])
data1_onehot["AnimalID"] = data1["AnimalID"]
data1 = pd.merge(data1_onehot, data1_copy, on='AnimalID',how='outer')
data1.outcome = data1.outcome.astype("int")
data1.outcome = data1.outcome.astype("object")
data1.outcomeG = data1.outcomeG.astype("int")
data1.outcomeG = data1.outcomeG.astype("object")
data1.to_csv("/Users/derri/Onedrive/ml/train2.csv")